In [2]:
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import glob
import pandas as pd
from IPython.display import display

In [97]:
#importing data from txt
#all outputs from log should be on same line (i.e. every line starts with a time stamp)
dirc = os.getcwd()
data = pd.DataFrame()
filenames = glob.glob(dirc+'/*.txt')
positions = []
mm_order = []
size = []
timestamp = []
price = []
spread = []
for file in filenames:
    f = open(file,"r")
    for line in f:
        #removes first few lines on text
        if re.search(r'(^\d*0)', line) == None:
            print(line)
            continue
        try:
            timestamp.append(int(re.search(r'(^\d*0)', line).group(1)))
        except:
            print('failed')
            
        if 'position: ' in line:
            positions.append(int(re.search(r'position: (-?\d*)', line).group(1)))
            if 'MM' in line:
                mm_order.append('MM')
                size.append(str(re.findall(r'(\d*.?\d*)x', line)[0])+'/-'+str(re.findall(r'(\d*.?\d*)x', line)[1]))
                price.append(str(re.findall(r'x (\d*)', line)[0])+'/-'+str(re.findall(r'x (\d*)', line)[1]))
                spread.append(int(re.search(r'Spread:  *(\d*)', line).group(1)))
            elif 'MISSED' in line:
                mm_order.append('MISSED')
                spread.append(0)
                size.append(float(re.search(r'(\d*.?\d*)x', line).group(1)))
                price.append(float(re.search(r'x (\d*)', line).group(1)))
            else:
                spread.append(0)
                mm_order.append(re.search(r'(SELL|BUY)', line).group(1))
                if re.search(r'(SELL|BUY)', line).group(1) == 'SELL':
                    size.append(-float(re.search(r'(\d*.?\d*)x', line).group(1)))
                else:
                    size.append(float(re.search(r'(\d*.?\d*)x', line).group(1)))
                price.append(float(re.search(r'x (\d*)', line).group(1)))
        else:
            positions.append(np.nan)
            mm_order.append(np.nan)
            size.append(np.nan)
            price.append(np.nan)
            spread.append(0)

data['timestamp'] = timestamp
data['Position'] = positions
data['MM/order'] = mm_order
data['Size_available'] = size
data['Price'] = price
data['Spread'] = spread
data2 = data.dropna()

print(data2)
#print(data)
#print(data.describe())

Sandbox logs:

START RequestId: d705ca66-ad61-4b63-8cdc-cbdb086c934d Version: $LATEST

Spread:  7 MM BUY PEARLS 5.0x 9999 position: 0 MM SELL PEARLS 5.0x 10004 position: 0

END RequestId: d705ca66-ad61-4b63-8cdc-cbdb086c934d

REPORT RequestId: d705ca66-ad61-4b63-8cdc-cbdb086c934d	Duration: 15.36 ms	Billed Duration: 16 ms	Memory Size: 128 MB	Max Memory Used: 65 MB	Init Duration: 543.33 ms	







Submission logs:

     timestamp  Position MM/order Size_available        Price  Spread
1          100      -1.0     SELL           -1.0      10002.0       0
2          200      -1.0       MM       5.0/-4.0  9997/-10003       8
3          300      -2.0       MM       5.0/-4.0  9996/-10004      10
4          400      -2.0       MM       5.0/-4.0  9999/-10003       6
5          500      -2.0       MM       5.0/-4.0  9999/-10003       6
..         ...       ...      ...            ...          ...     ...
995      99500       0.0      BUY            3.0       9998.0       0
996      99600       0.

In [113]:
#algo never MM adn trades at same time
#display(data2)
data2['Pos_T-1'] = data2['Position'].shift()
data2['Pos_T-1'][1] = 0
data2['Change'] = data2['Position'] - data2['Pos_T-1']
## did MM trade?


#print(data2.columns)
mm = []
for i, *row in data2.itertuples():
    #(data2['MM/order'] == 'MM') and (data2['Change'] != 0)
    if row[2] == 'MM' and row[7] != 0:
        mm.append(True)
    else:
        mm.append(False)

data2['MMtrade?'] = mm

#print(data2)
print(data2['MMtrade?'].describe())
hits = data2['MMtrade?'].describe()['count']-data2['MMtrade?'].describe()['freq']
print(hits, ' MM hits')


count       940
unique        2
top       False
freq        883
Name: MMtrade?, dtype: object
57  MM hits


C:\Users\tommy\AppData\Local\Temp\ipykernel_7624\1033647853.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Pos_T-1'] = data2['Position'].shift()
C:\Users\tommy\AppData\Local\Temp\ipykernel_7624\1033647853.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Pos_T-1'][1] = 0
C:\Users\tommy\AppData\Local\Temp\ipykernel_7624\1033647853.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

In [121]:
#MM hit data
mask = (data2['MMtrade?'] == True)
data_mmhits = data2.loc[mask, :]
print(data_mmhits.describe())
print(data_mmhits)
## note spread is the spread before our MM

##might be useful to calculate a hit rate at each spread, woudl require more testing 
#then can maximise for profit (half) spread*hitrate

          timestamp   Position     Spread    Pos_T-1     Change
count     57.000000  57.000000  57.000000  57.000000  57.000000
mean   57973.684211   1.368421   7.701754   2.280702  -0.912281
std    28662.285613  10.408842   1.451168  11.098189   3.163862
min      300.000000 -19.000000   6.000000 -20.000000  -9.000000
25%    34000.000000  -6.000000   6.000000  -6.000000  -2.000000
50%    59000.000000   2.000000   8.000000   4.000000  -1.000000
75%    79600.000000   9.000000   8.000000  11.000000   2.000000
max    99800.000000  19.000000  10.000000  20.000000   7.000000
     timestamp  Position MM/order Size_available        Price  Spread  \
3          300      -2.0       MM       5.0/-4.0  9996/-10004      10   
55        5500      17.0       MM       0.0/-9.0  9997/-10003       8   
56        5600      15.0       MM       1.0/-8.0  9999/-10003       6   
69        6900      14.0       MM       1.0/-8.0  9997/-10003       8   
104      10400       0.0       MM       5.0/-5.0  9996/-100

In [ ]:
#useful to plot data with dots when we make a trade (mm or order) diff colour

plt.plot(best_bid, label = asset, linewidth = 0.5, color = 'green')
plt.plot(best_ask , label = asset, linewidth = 0.5, color = 'red')
plt.plot(mid , label = "mid", linewidth = 0.5, color = "blue")
plt.plot(data1['FV'] , label = "orderbook fv", linewidth = 0.5, color = "black")
plt.plot(data1['FV_vol'] , label = "vol fv", linewidth = 0.5, color = "purple")
plt.plot(mid_average , label = str(periods)+"rolling average", linewidth = 0.5, color = "orange")
#plots dots
#for i, *row in data1.itertuples():   
    #check for large change 
#                 if row[21] >= 2 or row[21] <= -2:
#                     #print(row, row[21])
#                     plt.plot(i,row[15], 'ro')
plt.xlabel('Time')
plt.ylabel('Midprice')
plt.legend()
plt.title("Smoothed Prices " + asset + " Day " + day)
plt.show()